<a href="https://colab.research.google.com/github/Kussil/Financial_Sentiment_LLM/blob/main/05_Create_Vector_DB/Create_Vector_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create Vector Database

In [ ]:
#installs
!pip install tqdm
!pip install -q sentence_transformers
!pip install -q pinecone


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 6.3 MB/s eta 0:00:00


In [ ]:
#imports
from tqdm import tqdm
import os
from google.colab import userdata
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from pinecone import Pinecone, PodSpec, ServerlessSpec

### Import Files From GitHub

In [ ]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/CVX_Rice_project.git

Cloning into 'CVX_Rice_project'...
remote: Enumerating objects: 250, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 250 (delta 29), reused 40 (delta 21), pack-reused 200
Receiving objects: 100% (250/250), 97.74 MiB | 10.92 MiB/s, done.
Resolving deltas: 100% (129/129), done.
Updating files: 100% (22/22), done.


In [ ]:
url =  '/content/CVX_Rice_project/03_Sentiment_Analysis/sentiment_chunkdata.csv'
df_chunk = pd.read_csv(url)
df_chunk

,Unnamed: 0,Source,Unique_ID,Ticker,Date,Article Headline,URL,Article Index,Neutral,Positive,Negative,Text Chunks
0,0,Investment Research,NaN,CVX,2024-05-13,Chevron Corporation,NaN,0,0.467049,0.053343,0.479609,"[CLS] stock report | march 12, 2022 | nyse sym..."
1,1,Investment Research,NaN,CVX,2024-05-13,Chevron Corporation,NaN,0,0.467049,0.053343,0.479609,", 865 2018 35, 968 40, 491 42, 105 40, 338 158..."
2,2,Investment Research,NaN,CVX,2024-05-13,Chevron Corporation,NaN,0,0.467049,0.053343,0.479609,capital to dividends and buybacks. urisks to o...
3,3,Investment Research,NaN,CVX,2024-05-13,Chevron Corporation,NaN,0,0.467049,0.053343,0.479609,specific person who may receive this report. i...
4,4,Investment Research,NaN,CVX,2024-05-13,CFRA LIFTS VIEW ON SHARES OF CHEVRON CORPORATI...,NaN,1,0.178236,0.164126,0.657638,"[CLS] stock report | august 05, 2023 | nyse sy..."
...,...,...,...,...,...,...,...,...,...,...,...,...
14420,14420,ProQuest,2.464604e+09,XOM,2020-11-27,Why Exxons Cash Flow Still Wont Cover Whole,https://www.proquest.com/newspapers/why-exxons...,4519,0.118789,0.815717,0.065494,% from its close on wednesday. exxon shares ar...
14421,14421,ProQuest,2.464542e+09,XOM,2020-11-27,Exxon Internally Shows More Pessimism On Oil,https://www.proquest.com/newspapers/exxon-inte...,4520,0.011564,0.956691,0.031746,[CLS] exxon mobil corp. has lowered its outloo...
14422,14422,ProQuest,2.464542e+09,XOM,2020-11-27,Exxon Internally Shows More Pessimism On Oil,https://www.proquest.com/newspapers/exxon-inte...,4520,0.011564,0.956691,0.031746,##investment in oil and gas production would l...
14423,14423,ProQuest,2.464542e+09,XOM,2020-11-27,Exxon Internally Shows More Pessimism On Oil,https://www.proquest.com/newspapers/exxon-inte...,4520,0.011564,0.956691,0.031746,years argued that the company is extremely con...


### Encode Chunks

In [ ]:
#create text embeddings of chunks
embedding_model = SentenceTransformer("all-mpnet-base-v2")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedding_model.to(device)
text_embeddings = embedding_model.encode(df_chunk['Text Chunks'].tolist())
text_embeddings.shape

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(14425, 768)

### Create Vector Database

In [ ]:
#create vector
vectors = []
for idx in range(text_embeddings.shape[0]):
  vector = [float(i) for i in list(text_embeddings[idx])]
  vectors.append({"id": str(idx), "metadata": {"date": str(df_chunk['Date'][idx]),
                  "neu": float(df_chunk['Neutral'][idx]), "pos": float(df_chunk['Positive'][idx]),
                  "neg": float(df_chunk['Negative'][idx]), "ticker": df_chunk['Ticker'][idx]},
                  "values": vector })

#vectors[text_embeddings.shape[0]-1]

In [ ]:
#delete existing index to rebuild with new data
pc = Pinecone(api_key="bc4ea65c-d63e-48e4-9b65-53d6272d927d")

#pc.delete_index("newsembedding2")

In [ ]:
#create vector database index
pc = Pinecone(api_key="bc4ea65c-d63e-48e4-9b65-53d6272d927d")

index_name = "newsembedding3"

pc.create_index(
    name=index_name,
    dimension=text_embeddings.shape[1], # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'x-pinecone-api-version': '2024-04', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': 'dd2a2aad2bf0278749ab8730a42b9d77', 'Date': 'Wed, 22 May 2024 05:45:34 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
# insert vectors into database
index_name = "newsembedding3"

index = pc.Index(index_name)

# segments set to batch into vector database because of 2MB send limit
segments = 20
sections = int(text_embeddings.shape[0] / segments)
for i in tqdm (range(sections+1), desc="Loading..."):
  index.upsert(
    vectors=vectors[segments*i:segments*(1+i)],
    namespace="ns1"
  )

Loading...: 100%|██████████| 722/722 [04:30<00:00,  2.66it/s]


In [ ]:
#check index
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 16085}},
 'total_vector_count': 16085}

In [ ]:
#from google.colab import runtime
#runtime.unassign()